In [26]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd

from newrow import newrow_cash, newrow_etf_stock
from utils.fetch_utils import fetch_exchange_rate
import requests
from datetime import date, timedelta, datetime
import json
import yfinance as yf
from utils.other_utils import round_half_up


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# Un altro modo per ottenere il prezzo di chiusura più recente
# Scarica i dati degli ultimi due giorni con un intervallo di un minuto
latest_data = yf.download("USDEUR=X", period="2d", interval="1m", progress=False)

# Ottieni l'ultimo prezzo di chiusura
latest_price = latest_data['Close'].iloc[-1]
print(f"Ultimo prezzo di chiusura EUR/USD: {latest_price}")

Ultimo prezzo di chiusura EUR/USD: Ticker
USDEUR=X    0.8591
Name: 2025-10-15 15:28:00+00:00, dtype: float64
0.8590999841690063


In [4]:
# Un altro modo per ottenere il prezzo di chiusura più recente
# Scarica i dati degli ultimi due giorni con un intervallo di un minuto
latest_data = yf.download("USDEUR=X",start="2025-10-15", end="2025-10-16", progress=False)

# Ottieni l'ultimo prezzo di chiusura
latest_price = latest_data['Close'].iloc[-1]
print(f"Ultimo prezzo di chiusura EUR/USD: {latest_price[0]}")

Ultimo prezzo di chiusura EUR/USD: 0.8618999719619751


In [ ]:
dati = {}

dati[1] = "Fineco"

with open("reports/brokers.json", 'w', encoding='utf-8') as f:
    json.dump(dati, f, indent=4)

<class 'int'>


In [32]:
dati = {
    1: "Fineco",
    2: "BG Saxo"
}

print(dati["2"])

KeyError: '2'